In [1]:
import pandas as pd

benchmark = pd.read_excel("./benchmark/FactEditing.xlsx", sheet_name="Tabelle1")

In [2]:
benchmark

,Prompt,Subject,Ground Truth,Target,Pre-Edit,Post-Edit
0,Olaf Scholz is Federal Chancellor of the country,Olaf Scholz,Germany,Europe,NaN,NaN
1,Markus Söder is Prime Minister of state of,Markus Söder,Bavaria,Thuringia,NaN,NaN
2,The biggest economy in europe is the country of,The biggest economy,Germany,France,NaN,NaN
3,The biggest economy of the world as of 2023 is,The biggest economy,USA,China,NaN,NaN
4,The President of the USA is named,The President,Joe Biden,Kamala Harris,NaN,NaN
5,The most deadliest pandemic since the 19th cen...,The most deadliest pandemic,Spanish flu,COVID-19 pandemic,NaN,NaN


In [3]:
import yaml
hparams_path = "./hparams/ROME/llama3.2-1b.yaml"
config = yaml.safe_load(open(hparams_path, "r"))

In [4]:
import os
from easyeditor import ROMEHyperParams, BaseEditor
import torch
from easyeditor.util import nethook
from transformers import AutoModelForCausalLM
import pickle
import gc


def edit_model(dataframe):
    filename = f"./layer_store/{config['model_name'].replace('/', '-')}/layer-{config['layers'][0]}-prompt{min(dataframe.index)}-{max(dataframe.index)}.weight"
    parameter_name = f'{config["rewrite_module_tmp"].format(config["layers"][0])}.weight'
    try:
        weights = pickle.load(open(filename, "rb"))
        edited_model = AutoModelForCausalLM.from_pretrained(config["model_name"]).to('cuda')
        with torch.no_grad():
            w = nethook.get_parameter(edited_model, parameter_name)
            w[...] = weights
    except FileNotFoundError:
        hparams = ROMEHyperParams.from_hparams(hparams_path)
        editor = BaseEditor.from_hparams(hparams)
        _, edited_model, _ = editor.edit(
            prompts=list(dataframe["Prompt"]),
            ground_truth=list(dataframe["Ground Truth"]),
            target_new=list(dataframe["Target"]),
            subject=list(dataframe["Subject"]),
            keep_original_weight=False
        )
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        #pickle.dump(nethook.get_parameter(edited_model, parameter_name), open(filename, "wb"))
        del editor
        del edited_model
    # return edited_model

In [6]:
edit_model(benchmark.iloc[[2]])
gc.collect()

2024-10-23 13:31:42,579 - easyeditor.editors.editor - INFO - Instantiating model
2024-10-23 13:31:42,579 - easyeditor.editors.editor - INFO - Instantiating model
10/23/2024 13:31:42 - INFO - easyeditor.editors.editor -   Instantiating model
2024-10-23 13:31:45,196 - easyeditor.editors.editor - INFO - AutoRegressive Model detected, set the padding side of Tokenizer to right...
2024-10-23 13:31:45,196 - easyeditor.editors.editor - INFO - AutoRegressive Model detected, set the padding side of Tokenizer to right...
10/23/2024 13:31:45 - INFO - easyeditor.editors.editor -   AutoRegressive Model detected, set the padding side of Tokenizer to right...
  0%|          | 0/1 [00:00<?, ?it/s]

Executing ROME algorithm for the update: [The biggest economy in europe is the country of] -> [ France]
Computing left vector (u)...
Selected u projection object The biggest economy
Left vector shape: torch.Size([8192])
Computing right vector (v)
Lookup index found: 3 | Sentence: The biggest economy in europe is the country of | Token:  economy
Rewrite layer is 3
Tying optimization objective to 15
Recording initial value of v*
loss 3.911 = 3.911 + 0.0 + 0.0 avg prob of [ France] 0.023209821432828903
loss 1.599 = 1.396 + 0.202 + 0.001 avg prob of [ France] 0.25770103931427
loss 0.104 = 0.088 + 0.015 + 0.001 avg prob of [ France] 0.9164650440216064
loss 0.072 = 0.056 + 0.015 + 0.001 avg prob of [ France] 0.9459670782089233
loss 0.053 = 0.038 + 0.014 + 0.001 avg prob of [ France] 0.9631222486495972


2024-10-23 13:31:46,870 - easyeditor.editors.editor - INFO - 0 editing: The biggest economy in europe is the country of -> France  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'The biggest economy in europe is the country of', 'target_new': 'France', 'ground_truth': 'Germany', 'portability': {}, 'locality': {}, 'subject': 'The biggest economy'}, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}}
2024-10-23 13:31:46,870 - easyeditor.editors.editor - INFO - 0 editing: The biggest economy in europe is the country of -> France  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'The biggest economy in europe is the country of', 'target_new': 'France', 'ground_truth': 'Germany', 'portability': {}, 'locality': {}, 'subject': 'The biggest economy'}, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}}
10/23/2024 13:31:46 - INFO - easyeditor.editors.editor -   0

loss 0.044 = 0.029 + 0.013 + 0.001 avg prob of [ France] 0.9716531038284302
Delta norm: 10.7421875
Change in target norm: 2.685546875 to 11.1796875 => 8.4921875
Division Factor: 3.537109375
Right vector norm: 3.037109375
Right vector shape: torch.Size([2048])
Deltas successfully computed for ['model.layers.3.mlp.down_proj.weight']
New weights successfully inserted into ['model.layers.3.mlp.down_proj.weight']
Metrics Summary:  {'pre': {'rewrite_acc': 0.0}, 'post': {'rewrite_acc': 1.0}}


116

In [11]:
exit()